In [4]:
import pandas as pd
import requests
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Dict
import time

In [6]:
def fetch_api_data(
    id: str,
    url: str = "https://stats-royale-api-js-beta-z2msk5bu3q-uk.a.run.app/profile/",
    headers: Dict = None,
) -> Dict:
    """Fetch data from a single API endpoint"""
    try:
        request_url = f"{url}{id}"
        print(f"Fetching: {request_url}")
        response = requests.get(request_url, headers=headers, timeout=10)
        response.raise_for_status()
        return {"id": id, "data": response.json(), "status": "success"}
    except requests.exceptions.RequestException as e:
        return {"id": id, "data": None, "status": "failed", "error": str(e)}


def parallel_api_calls(
    ids: List[str], max_workers: int = 5, headers: Dict = None
) -> pd.DataFrame:
    """
    Make parallel API calls

    Args:
        urls: List of API URLs to call
        max_workers: Number of parallel workers (default: 5)
        headers: Optional headers for API requests

    Returns:
        List of results from all API calls
    """
    df = pd.DataFrame()

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks
        future_to_id = {executor.submit(fetch_api_data, id): id for id in ids}
        results = []
        # Process completed tasks
        for future in as_completed(future_to_id):
            result = future.result()
            results.append(result)
            print(f"Completed: {result['id']} - Status: {result['status']}")

    df = pd.DataFrame(results)
    return df

def save_to_csv(data: pd.DataFrame, filename: str):
    """Save DataFrame to CSV file"""
    data.to_csv(filename, index=False)
    print(f"Data saved to {filename}")
# Example usage:
# Define your Player ids
ids = ["2G2L9PRUP"]

# Optional: Add headers if needed (e.g., API key)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36",
}

# Make parallel API calls
results_df = parallel_api_calls(ids, max_workers=10, headers=headers)

# Save to CSV
save_to_csv(results_df, "battle_log_data.csv")


Fetching: https://stats-royale-api-js-beta-z2msk5bu3q-uk.a.run.app/profile/2G2L9PRUP
Completed: 2G2L9PRUP - Status: success
Data saved to battle_log_data.csv
